# Plot figures for the DNAscope LongRead paper

In [ ]:
work_dir = "/work"
dnascope_output_format = "dnascope/{ref}/{sample}/{aln_params}/{subset}/calls.vcf.gz"
happy_merged_output = "happy_merged/happy_extended_metrics.tsv"
config_file = "config.yaml"

In [ ]:
from collections import namedtuple
import math
import os
import os.path
import re

import matplotlib.pyplot as plt
# %matplotlib inline
import matplotlib as mpl
import pandas as pd
import seaborn as sns
import yaml

In [ ]:
# Load the config file
config = yaml.load(open(config_file), Loader=yaml.CLoader)

### Utility functions

In [ ]:
def iter_processed_samples():
    """ A generator over the processed samples """
    d = {
        "ref": "hs38", "aln_params": "--preset HiFi -c 0 -y 70"
    }
    
    # All samples with full coverage
    d["truthset"] = "v4.2.1"
    d["subset"] = "full"
    for sample in config["input"]["samples"].keys():
        d["sample"] = sample
        yield d.copy()
    
    # Downsamples of HG003
    d["sample"] = "pFDA-truth_V2-PB-HG003"
    for subset in config["pipeline"]["hg003_subsets"]:
        d["subset"] = subset
        yield d.copy()
    
    # pFDA HG002 with CMRG truthset
    d["truthset"] = "CMRGv1.00"
    d["sample"] = "pFDA-truth_V2-PB-HG002"
    d["subset"] = "full"
    yield d.copy()

In [ ]:
def update_row_values(col, _dict):
    """ A small utility function for updating values in a pandas DataFrame using a dict to map values """
    def _update_row_values(row):
        val = row[col]
        if val not in _dict:
            return val
        else:
            return _dict[row[col]]
    return _update_row_values

In [ ]:
def happy_to_errors(df):
    """ Convert hap.py rows to a table with errors for plotting """
    new_df = {
        "Errors": [], "Variant Type": [], "Error": [],
    }
    for idx, row in df.iterrows():
        error_types = ("TRUTH.FN", "QUERY.FP")
        error_names = ("False Negative", "False Positive")
        for error_type, name in zip(error_types, error_names):
            new_df["Errors"].append(row[error_type])
            new_df["Variant Type"].append(row["Type"])
            new_df["Error"].append(name)
    return pd.DataFrame(new_df)

def happy_to_acc(df):
    """ Convert hap.py rows to a table with accuracy metrics for plotting """
    new_df = {
        "Performance": [], "Variant Type": [], "Metric": [],
    }
    for idx, row in df.iterrows():
        metric_types = ("METRIC.Recall", "METRIC.Precision")
        metric_names = ("Recall", "Precision")
        for metric_type, name in zip(metric_types, metric_names):
            new_df["Performance"].append(row[metric_type])
            new_df["Variant Type"].append(row["Type"])
            new_df["Metric"].append(name)
    return pd.DataFrame(new_df)

## Read input data

### Collect the runtime for each sample

In [ ]:
sample_runtime_metrics = {}
StageRuntime = namedtuple("StageRuntime", ["memory", "user", "sys", "real"])
runtime_stages = [
    "RepeatStat", "DNAscope-diploid", "DNAModelApply-diploid", "VariantPhaser", 
    "RepeatModel", "DNAscope-hap1", "DNAscope-hap2", "DNAModelApply-hap1",
    "DNAModelApply-hap2", "DNAscopeHP-unphased", "DNAModelApply-unphased",
]
for wildcards in iter_processed_samples():
    # Skip the CMRG iteration - runtime is the same as v4.2.1
    if wildcards["truthset"] == "CMRGv1.00":
        continue
    
    sample_metrics = {}
    coverage_name = "full"
    if wildcards["subset"] != "full":
        downsample_frac = float(f"0.{wildcards['subset']}")
        downsample_target = int(downsample_frac * 36.0)
        coverage_name = f"{downsample_target}x"
    sample_name = f"{wildcards['sample']}--{coverage_name}"
    
    dnascope_vcf_fn = os.path.join(work_dir, dnascope_output_format.format(**wildcards))
    stdout_fn, stderr_fn, benchmark_fn = (
        dnascope_vcf_fn + ".stdout", 
        dnascope_vcf_fn + ".stderr", 
        dnascope_vcf_fn + ".benchmark.txt",
    )
    
    # Collect the memory usage from the ".benchmark.txt" file. Runtime and io 
    # metrics will not be accurate from this file due to the input BAM being copied
    # to a local SSD inside this rule
    benchmark_metrics = pd.read_csv(benchmark_fn, sep='\t', header=0)
    sample_metrics["max_vms"] = float(benchmark_metrics.at[0, "max_vms"])
    
    # Overall runtime is written to stdout
    start_time, end_time = None, None
    with open(stdout_fn) as stdout_fh:
        for line in stdout_fh:
            line = line.rstrip()
            if line.startswith("Start time: "):
                if start_time is not None:
                    raise ValueError(f"Found multiple start times in file, '{stdout_fn}'")
                start_time = int(line[12:])
            if line.startswith("End time: "):
                if end_time is not None:
                    raise ValueError(f"Found multiple end times in file, '{stdout_fn}'")
                end_time = int(line[10:])
    
    if not start_time or not end_time:
        raise ValueError(f"Failed to find start or end time in file, '{stdout_fn}'")
    sample_metrics["overall_runtime"] = end_time - start_time
    
    # Per-stage runtime is written to stderr in a pre-determined order
    stage_metrics = []
    with open(stderr_fn) as stderr_fh:
        for line in stderr_fh:
            if not line.startswith("overall: "):
                continue
            line = line.rstrip().split(' ')
            stage_metrics.append(
                StageRuntime(
                    int(line[1]),
                    float(line[3]),
                    float(line[5]),
                    float(line[7]),
                )
            )
    # Find runtime that is not in the per-stage runtime
    accounted_runtime = sum([x.real for x in stage_metrics])
    other_runtime = sample_metrics["overall_runtime"] - accounted_runtime
    
    sample_metrics.update(dict(zip(runtime_stages, stage_metrics)))
    sample_metrics["other_runtime"] = other_runtime
    sample_runtime_metrics[sample_name] = sample_metrics

### Read the aggregated hap.py metrics

In [ ]:
merged_happy_fn = os.path.join(work_dir, happy_merged_output)
accuracy_metrics = pd.read_csv(merged_happy_fn, sep='\t', header=0)

In [ ]:
# Build rows for combined SNVs and INDELs
sample_subset = {}
for idx, row in accuracy_metrics.iterrows():
    _subset_sample = (row["Subset"], row["Sample"])
    
    # initialize the sample/subset, if necessary
    if _subset_sample not in sample_subset:
        sample_subset[_subset_sample] = [0, 0, 0, 0]
    
    # Add the new data to the earlier data
    row_data = list(row[["TRUTH.TOTAL", "TRUTH.TP", "TRUTH.FN", "QUERY.FP"]])
    sample_subset[_subset_sample] = [sum(x) for x in zip(sample_subset[_subset_sample], row_data)]

# Build a new DF with SNV+INDEL
new_df = {
    "Type": [], "Subset": [], "METRIC.Recall": [], "METRIC.Precision": [],
    "METRIC.F1_Score": [], "TRUTH.TOTAL": [], "TRUTH.TP": [], "TRUTH.FN": [],
    "QUERY.FP": [], "TOTAL.ERRORS": [], "Sample": [],
}
for _subset_sample, metrics in sample_subset.items():
    subset, sample = _subset_sample
    total, tp, fn, fp = metrics
    
    new_df["Type"].append("SNP+INDEL")
    new_df["Subset"].append(subset)
    new_df["Sample"].append(sample)
    
    try:
        recall = float(tp) / total
    except ZeroDivisionError:
        recall = 0.0
    
    try:
        precision = float(tp) / (tp + fp)
    except ZeroDivisionError:
        precision = 0.0
    
    try:
        f1_score = 2 * recall * precision / (recall + precision)
    except ZeroDivisionError:
        f1_score = 0.0
    
    new_df["METRIC.Recall"].append(recall)
    new_df["METRIC.Precision"].append(precision)
    new_df["METRIC.F1_Score"].append(f1_score)
    new_df["TRUTH.TOTAL"].append(total)
    new_df["TRUTH.TP"].append(tp)
    new_df["TRUTH.FN"].append(fn)
    new_df["QUERY.FP"].append(fp)
    new_df["TOTAL.ERRORS"].append(fn + fp)

new_df = pd.DataFrame(data=new_df)

# Concat the DFs
accuracy_metrics = pd.concat([accuracy_metrics, new_df])

## Generate figures
### Runtime by Stage

In [ ]:
# Build an empty dict with the columns
runtime_df = {"sample":[]}
plot_stages = runtime_stages + ["other_runtime"]
for stage in plot_stages:
    runtime_df[stage] = []

# Add the data
for sample, metrics in sample_runtime_metrics.items():
    runtime_df["sample"].append(sample)
    for stage in plot_stages:
        value = metrics[stage]
        if isinstance(value, float):
            runtime_df[stage].append(metrics[stage])
        else:
            runtime_df[stage].append(metrics[stage].real)
runtime_df = pd.DataFrame(data=runtime_df)

# Use better names
remap_names = {
    'pFDA-truth_V2-PB-HG003--5x': "pFDA-HG3-5x",
    'pFDA-truth_V2-PB-HG003--7x': "pFDA-HG3-7x",
    'pFDA-truth_V2-PB-HG003--8x': "pFDA-HG3-8x",
    'pFDA-truth_V2-PB-HG003--9x': "pFDA-HG3-9x",
    'pFDA-truth_V2-PB-HG003--10x': "pFDA-HG3-10x",
    'pFDA-truth_V2-PB-HG003--11x': "pFDA-HG3-11x",
    'pFDA-truth_V2-PB-HG003--13x': "pFDA-HG3-13x",
    'pFDA-truth_V2-PB-HG003--15x': "pFDA-HG3-15x",
    'pFDA-truth_V2-PB-HG003--17x': "pFDA-HG3-17x",
    'pFDA-truth_V2-PB-HG003--20x': "pFDA-HG3-20x",
    'pFDA-truth_V2-PB-HG003--25x': "pFDA-HG3-25x",
    'pFDA-truth_V2-PB-HG003--30x': "pFDA-HG3-30x",
    'pFDA-truth_V2-PB-HG003--full': "pFDA-HG3-full",
    'pFDA-truth_V2-PB-HG004--full': "pFDA-HG4-full",
    'pFDA-truth_V2-PB-HG002--full': "pFDA-HG2-full",
    'HG002-chemV2.2-sample--full': "chemV2.2-HG2-full",
}
runtime_df["sample"] = runtime_df.apply(update_row_values("sample", remap_names), axis=1)

# Skip samples not run under benchmark conditions
skip_samples = [
    "pFDA-HG3-11x",
    "pFDA-HG3-13x",
    "pFDA-HG3-17x",
]
runtime_df = runtime_df.loc[
    (~runtime_df["sample"].isin(skip_samples))
]

# Re-order the columns
sample_order = [
    "pFDA-HG3-5x",
    "pFDA-HG3-10x",
    "pFDA-HG3-15x",
    "pFDA-HG3-20x",
    "pFDA-HG3-25x",
    "pFDA-HG3-30x",
    "pFDA-HG3-full",
    "pFDA-HG4-full",
    "pFDA-HG2-full",
    "chemV2.2-HG2-full",
]
runtime_df = runtime_df.set_index("sample").reindex(sample_order)

In [ ]:
# Convert the runtime metrics from seconds to hours
for ridx, row in runtime_df.iterrows():
    for cidx, val in row.items():
        row[cidx] = val / 60 / 60

In [ ]:
# Print some runtime metrics

print("HG003 5x sample runtime:")
print(runtime_df.loc["pFDA-HG3-5x"].sum())
print("")

print("HG002 ChemV2.2 sample runtime:")
print(runtime_df.loc["chemV2.2-HG2-full"].sum())
print("")

print("ChemV2.2 first pass runtime:")
print(runtime_df[
    ['RepeatStat', 'DNAscope-diploid', 'DNAModelApply-diploid']
].loc["chemV2.2-HG2-full"].sum())
print("")

print("ChemV2.2 phasing runtime:")
print(runtime_df[
    ['VariantPhaser', 'RepeatModel']
].loc["chemV2.2-HG2-full"].sum())
print("")

print("ChemV2.2 second pass runtime:")
print(runtime_df[
    ['DNAscope-hap1', 'DNAscope-hap2', 'DNAModelApply-hap1', 'DNAModelApply-hap2', 
     'DNAscopeHP-unphased', 'DNAModelApply-unphased']
].loc["chemV2.2-HG2-full"].sum())
print("")

print("ChemV2.2 other runtime:")
print(runtime_df[
    ['other_runtime']
].loc["chemV2.2-HG2-full"].sum())

In [ ]:
runtime_df

In [ ]:
runtime_df.to_csv(
    "../analysis/benchmark_runtime.tsv",
    sep="\t",
    float_format="%.5f",
)

In [ ]:
runtime_colors = [
    (0.109, 0.494, 0.792), # RepeatStat
    (0.047, 0.392, 0.513), # DNAscope-diploid
    (0.062, 0.203, 0.776), # DNAModelApply-diploid
    (0.964, 0.074, 0.156), # VariantPhaser
    (0.964, 0.298, 0.074), # RepeatModel
    (0.705, 0.972, 0.105), # DNAscope-hap1
    (0.415, 0.972, 0.105), # DNAscope-hap2
    (0.286, 0.874, 0.203), # DNAModelApply-hap1
    (0.156, 0.784, 0.235), # DNAModelApply-hap2
    (0.278, 0.980, 0.701), # DNAscopeHP-unphased
    (0.380, 0.980, 0.823), # DNAModelApply-unphased
    (0.898, 0.482, 0.043), # other_runtime
]

# Plot
ax = runtime_df.plot(
    kind="bar",
    stacked=True,
    rot=30,
    xlabel="Sample",
    ylabel="Runtime (hr)",
    color=runtime_colors,
)
ax.legend(bbox_to_anchor=(1.05, 1))
plt.close(2)
plt.show()

### Memory usage by sample

In [ ]:
# Build an empty dict with the columns
memory_df = {"sample":[], "max_vms": []}

# Add the data
for sample, metrics in sample_runtime_metrics.items():
    memory_df["sample"].append(remap_names[sample])
    memory_df["max_vms"].append(metrics["max_vms"] / 1024) # Memory from MB to GB
    
memory_df = pd.DataFrame(data=memory_df)

# Skip samples not run under benchmark conditions
memory_df = memory_df.loc[
    (~memory_df["sample"].isin(skip_samples))
]

In [ ]:
memory_df

In [ ]:
memory_df.to_csv(
    "../analysis/benchmark_memory.tsv",
    sep="\t",
    index=False,
    float_format="%.5f",
)

In [ ]:
ax = sns.barplot(
    x="sample",
    y="max_vms",
    data=memory_df,
    order=sample_order,
)
ax.set(ylabel="Maximum Memory (GB)", xlabel="Sample")
ax.set_xticklabels(ax.get_xticklabels(), rotation=30)

### Accuracy vs. the pFDA Truth Challenge V2 results

In [ ]:
# Subset the full accuracy data frame
regions_map = {
    "*": "All Benchmark Regions",
    "GRCh38_MHC.bed.gz": "MHC",
    "GRCh38_alllowmapandsegdupregions.bed.gz": "Difficult-to-Map Regions",
}
pfda_samples = {
    "pFDA-truth_V2-PB-HG002--full--v4.2.1": "HG002",
    "pFDA-truth_V2-PB-HG003--full--v4.2.1": "HG003",
    "pFDA-truth_V2-PB-HG004--full--v4.2.1": "HG004",
    "GH_pFDA_truth_V2--HG002--full--v4.2.1": "HG002",
    "GH_pFDA_truth_V2--HG003--full--v4.2.1": "HG003",
    "GH_pFDA_truth_V2--HG004--full--v4.2.1": "HG004",
}
pfda_caller = {
    "pFDA-truth_V2-PB-HG002--full--v4.2.1": "DNAscope LongRead",
    "pFDA-truth_V2-PB-HG003--full--v4.2.1": "DNAscope LongRead",
    "pFDA-truth_V2-PB-HG004--full--v4.2.1": "DNAscope LongRead",
    "GH_pFDA_truth_V2--HG002--full--v4.2.1": "pFDA Truth V2 Winning",
    "GH_pFDA_truth_V2--HG003--full--v4.2.1": "pFDA Truth V2 Winning",
    "GH_pFDA_truth_V2--HG004--full--v4.2.1": "pFDA Truth V2 Winning",
}

pfda_acc_df = accuracy_metrics.loc[
    (accuracy_metrics["Sample"].isin(list(pfda_samples.keys()))) &
    (accuracy_metrics["Subset"].isin(list(regions_map.keys()))) &
    (accuracy_metrics["Type"] == "SNP+INDEL")
].copy()

In [ ]:
pfda_acc_df["Subset"] = pfda_acc_df.apply(update_row_values("Subset", regions_map), axis=1)
pfda_acc_df["Caller"] = pfda_acc_df.apply(update_row_values("Sample", pfda_caller), axis=1)
pfda_acc_df["Sample"] = pfda_acc_df.apply(update_row_values("Sample", pfda_samples), axis=1)

In [ ]:
pfda_acc_df.drop(
    ["Type", "TOTAL.ERRORS"],
    axis=1,
)

In [ ]:
# Accuracy table
pfda_acc_df.drop(
    ["Type", "TOTAL.ERRORS"],
    axis=1,
).to_csv(
    "../analysis/pfda_acc_table.tsv",
    sep="\t",
    index=False,
    float_format="%.5f",
)

In [ ]:
# Print some accuracy metrics

# pFDA average accuracy
winning_mean = pfda_acc_df.loc[
    (pfda_acc_df["Subset"] == "All Benchmark Regions") &
    (pfda_acc_df["Caller"] == "pFDA Truth V2 Winning")
]["METRIC.F1_Score"].mean()

dnascope_mean = pfda_acc_df.loc[
    (pfda_acc_df["Subset"] == "All Benchmark Regions") &
    (pfda_acc_df["Caller"] == "DNAscope LongRead")
]["METRIC.F1_Score"].mean()

print(f"DNAscope mean accuracy: {dnascope_mean}")
print(f"Winning pipeline mean accuracy: {winning_mean}")

# pFDA mean errors by stratification
stratifications = ("All Benchmark Regions", "MHC", "Difficult-to-Map Regions")
callers = ("pFDA Truth V2 Winning", "DNAscope LongRead")
for strat in stratifications:
    mean_errors = []
    for caller in callers:
        mean_errors.append(
            pfda_acc_df.loc[
                (pfda_acc_df["Subset"] == strat) &
                (pfda_acc_df["Caller"] == caller)
            ]["TOTAL.ERRORS"].mean()
        )
    
    mean_err_red = (mean_errors[0] - mean_errors[1]) / mean_errors[0]
    print(f"Mean error reduction across '{strat}' is: {mean_err_red}")

In [ ]:
# Percent errors relative to DNAscope LongReads
pfda_acc_barplot = pfda_acc_df.loc[
    pfda_acc_df["Sample"] == "HG003"
].copy()

dnascope_errs_dict = {}
for idx, row in pfda_acc_barplot.loc[
    pfda_acc_barplot["Caller"] == "DNAscope LongRead"
].iterrows():
    dnascope_errs_dict[row["Subset"]] = row["TOTAL.ERRORS"]

def relative_errors(row):
    return row["TOTAL.ERRORS"] / dnascope_errs_dict[row["Subset"]] * 100
    
pfda_acc_barplot["Relative Errors"] = pfda_acc_barplot.apply(relative_errors, axis=1)

# plot
ax = sns.barplot(x="Subset", y="Relative Errors", hue="Caller", data=pfda_acc_barplot)
ax.set(ylabel="% Errors Relative to DNAscope LongRead")
plt.close(2)
plt.show()

### Accuracy on serial downsamples of the pFDA HG003 dataset

In [ ]:
# Subset the full dataframe
homopolymer_regions_map = {
    "*": "All Benchmark Regions",
    "GRCh38_MHC.bed.gz": "MHC",
    "GRCh38_alllowmapandsegdupregions.bed.gz": "Difficult-to-Map Regions",
    "GRCh38_notinAllHomopolymers_gt6bp_imperfectgt10bp_slop5.bed.gz": "Not Long Homopolymer",
}
sample_depth = {
    "pFDA-truth_V2-PB-HG003--full--v4.2.1": 35,
    "pFDA-truth_V2-PB-HG003--857142--v4.2.1": 30,
    "pFDA-truth_V2-PB-HG003--714285--v4.2.1": 25,
    "pFDA-truth_V2-PB-HG003--571428--v4.2.1": 20,
    "pFDA-truth_V2-PB-HG003--485714--v4.2.1": 17,
    "pFDA-truth_V2-PB-HG003--428571--v4.2.1": 15,
    "pFDA-truth_V2-PB-HG003--371429--v4.2.1": 13,
    "pFDA-truth_V2-PB-HG003--314286--v4.2.1": 11,
    "pFDA-truth_V2-PB-HG003--285714--v4.2.1": 10,
    "pFDA-truth_V2-PB-HG003--142857--v4.2.1": 5,
}

hg003_downsample_df = accuracy_metrics.loc[
    (accuracy_metrics["Sample"].isin(list(sample_depth.keys()))) &
    (accuracy_metrics["Subset"].isin(list(homopolymer_regions_map.keys())))
].copy()

In [ ]:
hg003_downsample_df["Depth"] = hg003_downsample_df.apply(update_row_values("Sample", sample_depth), axis=1)
hg003_downsample_df["Subset"] = hg003_downsample_df.apply(update_row_values("Subset", homopolymer_regions_map), axis=1)

In [ ]:
hg003_downsample_df.loc[
    (
        (hg003_downsample_df["Subset"] == "All Benchmark Regions") |
        (hg003_downsample_df["Subset"] == "Not Long Homopolymer")
    ) &
    (hg003_downsample_df["Type"] != "SNP+INDEL")
].drop(
    ["TOTAL.ERRORS", "Sample"],
    axis=1,
)

In [ ]:
hg003_downsample_df.loc[
    (
        (hg003_downsample_df["Subset"] == "All Benchmark Regions") |
        (hg003_downsample_df["Subset"] == "Not Long Homopolymer")
    ) &
    (hg003_downsample_df["Type"] != "SNP+INDEL")
].drop(
    ["TOTAL.ERRORS", "Sample"],
    axis=1,
).to_csv(
    "../analysis/hg003_downsample_table.tsv",
    sep="\t",
    index=False,
    float_format="%.5f",
)

In [ ]:
# Add rows for Indel - no Homopolymer
def update_var_type(row):
    """ Update the variant type """
    if row["Type"] == "INDEL" and row["Subset"] == "Not Long Homopolymer":
        return "INDEL - not long Homopolymer"
    return row["Type"]

hg003_downsample_df["Type"] = hg003_downsample_df.apply(update_var_type, axis=1)

In [ ]:
# Plot F1-score by Depth
ax = sns.lineplot(
    y="METRIC.F1_Score",
    x="Depth",
    data=hg003_downsample_df.loc[
        (
            (hg003_downsample_df["Subset"] == "All Benchmark Regions") &
            (hg003_downsample_df["Type"] != "SNP+INDEL")
        ) | (
            (hg003_downsample_df["Type"] == "INDEL - not long Homopolymer")
        )
    ],
    hue="Type",
    style="Type",
    markers=True,
    dashes=False,
)
ax.set(ylabel="F1-Score")
plt.close(2)
plt.show()

In [ ]:
# Plot Phred(F1-score) by Depth
hg003_downsample_df["Phred_F1"] = hg003_downsample_df["METRIC.F1_Score"].map(lambda x: -10 * math.log10(1 - x))

ax = sns.lineplot(
    y="Phred_F1",
    x="Depth",
    data=hg003_downsample_df.loc[
        (
            (hg003_downsample_df["Subset"] == "All Benchmark Regions") &
            (hg003_downsample_df["Type"] != "SNP+INDEL")
        ) | (
            (hg003_downsample_df["Type"] == "INDEL - not long Homopolymer")
        )
    ],
    hue="Type",
    style="Type",
    markers=True,
    dashes=False,
)
ax.set(ylabel="Phred(F1-Score)")
plt.close(2)
plt.show()

### Accuracy on the CMRG benchmark dataset

In [ ]:
cmrg_accuracy = accuracy_metrics.loc[
    (accuracy_metrics["Sample"] == "pFDA-truth_V2-PB-HG002--full--CMRGv1.00") &
    (accuracy_metrics["Subset"] == "*") &
    (accuracy_metrics["Type"] != "SNP+INDEL")
]

In [ ]:
cmrg_errors = happy_to_errors(cmrg_accuracy)
ax = sns.barplot(x="Variant Type", y="Errors", hue="Error", data=cmrg_errors)
plt.close(2)
plt.show()

In [ ]:
cmrg_metrics = happy_to_acc(cmrg_accuracy)
ax = sns.barplot(x="Variant Type", y="Performance", hue="Metric", data=cmrg_metrics)
ax.set(ylim=(0.9, 1.003))
plt.close(2)
plt.show()

In [ ]:
cmrg_accuracy.drop(
    ["Subset", "Sample"],
    axis=1,
).to_csv(
    "../analysis/cmrg_table.tsv",
    sep="\t",
    index=False,
    float_format="%.5f",
)

### Accuracy on the Chemistry v2.2 sample

In [ ]:
chem_accuracy = accuracy_metrics.loc[
    (accuracy_metrics["Sample"] == "HG002-chemV2.2-sample--full--v4.2.1") &
    (accuracy_metrics["Subset"] == "*") &
    (accuracy_metrics["Type"] != "SNP+INDEL")
]

In [ ]:
chem_errors = happy_to_errors(chem_accuracy)
ax = sns.barplot(x="Variant Type", y="Errors", hue="Error", data=chem_errors)
plt.close(2)
plt.show()

In [ ]:
chem_metrics = happy_to_acc(chem_accuracy)
ax = sns.barplot(x="Variant Type", y="Performance", hue="Metric", data=chem_metrics)
ax.set(ylim=(0.98, 1.003))
plt.close(2)
plt.show()

In [ ]:
accuracy_metrics.loc[
    (accuracy_metrics["Sample"] == "HG002-chemV2.2-sample--full--v4.2.1") &
    (accuracy_metrics["Subset"] == "*")
]

In [ ]:
accuracy_metrics.loc[
    (accuracy_metrics["Sample"] == "HG002-chemV2.2-sample--full--v4.2.1") &
    (accuracy_metrics["Subset"] == "*")
].copy().drop(
    ["Subset", "Sample", "TOTAL.ERRORS"],
    axis=1,
).to_csv(
    "../analysis/chemistry-v2.2_table.tsv",
    sep="\t",
    index=False,
    float_format="%.5f",
)

### Accuracy on the Chem V2.2 sample vs. HG002 pFDA Truth V2 sample

In [ ]:
chem_pfda_samples = {
    "HG002-chemV2.2-sample--full--v4.2.1": "HG002 - chemV2.2",
    "pFDA-truth_V2-PB-HG002--full--v4.2.1": "HG002 - pFDA TruthV2",
}

chem_accuracy = accuracy_metrics.loc[
    (accuracy_metrics["Sample"].isin(list(chem_pfda_samples.keys()))) &
    (accuracy_metrics["Subset"] == "*") &
    (accuracy_metrics["Type"] != "SNP+INDEL")
].copy()

chem_accuracy["Sample"] = chem_accuracy.apply(update_row_values("Sample", chem_pfda_samples), axis=1)

In [ ]:
ax = sns.barplot(x="Type", y="METRIC.F1_Score", hue="Sample", data=chem_accuracy)
ax.set(ylim=(0.99, 1.001), xlabel="Variant Type", ylabel="F-score")
plt.close(2)
plt.show()